**Uploading and extracting Mytorch.zip**

In [ ]:
# upload Mytorch.zip file here
from google.colab import files
uploaded=files.upload()

In [ ]:
# extracts Mytorch
import zipfile
zip_file_name = 'Mytorch.zip'
extract_location = '/content/'

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_location)

**Imports from Mytroch:**

In [ ]:
import Mytorch
import Mytorch.Network.Network as Network
from Mytorch.ActivationFunctions import Sigmoid,Tanh,ReLU,LeakyReLU,Softmax
from Mytorch.Layers import FCLayer
from Mytorch.LossFunctions import BinaryCrossEntropy, CrossEntropy, MeanSquaredError
from Mytorch.Optimizers import GradientDescent, GradientDescentWithMomentum, RMSProp, Adam


**Creating Network:**

Network Archetecture:


```
fullyconnecedlayer(inputsize=784,output_size=100)
tanh activation layer()
fullyconnectedlayer(input_size=100,output_size=50)
tanh activation layer()
fullyconnectedlayer(input_size=50, output_size=10)
tanh activation layer()
```



In [ ]:
import numpy as np
net = Network()
net.add_layers(FCLayer(28*28, 100))
net.add_layers(Tanh())
net.add_layers(FCLayer(100,50))
net.add_layers(Tanh())
net.add_layers(FCLayer(50, 10))
net.add_layers(Softmax())

In [ ]:

#accuracy function
def accuracy(pred,true):
    return np.sum(true==pred,axis=0)/len(true)

**Loading MNIST handwritten digit dataset from pytorch**

we're using pytorch only to load dataset.



In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)



def to_one_hot(labels, num_classes=10):
    one_hot_labels = torch.zeros(labels.size(0), num_classes)
    for i, label in enumerate(labels):
        one_hot_labels[i][label] = 1.0
    return one_hot_labels

100%|██████████| 9912422/9912422 [00:00<00:00, 124086228.36it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 17018220.54it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 31586047.28it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5220753.29it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



**Training:**

In [ ]:


learning_rate= 0.001
optimizer=GradientDescent(net.layers,learning_rate=learning_rate)
error=BinaryCrossEntropy()
epochs=5
for epoch in range(epochs):
    epoch_loss=[]
    acc=[]
    for images,labels in train_loader:
        data=images.reshape(images.shape[0],1,28*28)
        targets=to_one_hot(labels)
        data=data.numpy()
        targets=targets.numpy()
        outputs=net.predict(data)
        loss=error.Loss(targets,outputs)
        epoch_loss.append(loss)
        acc.append(accuracy(np.argmax(outputs,axis=1),np.argmax(targets,axis=1)))
        error.backward(net,targets,outputs)
        optimizer.step()

    print(f"epoch:{epoch+1}, loss:{np.mean(epoch_loss)} , accuracy:{np.mean(acc)*100}%")

epoch:1, loss:0.05534357757459145 , accuracy:90.18666666666667%
epoch:2, loss:0.029554867894933914 , accuracy:94.87166666666667%
epoch:3, loss:0.02261124664236384 , accuracy:96.13166666666667%
epoch:4, loss:0.018698921618208766 , accuracy:96.79333333333334%
epoch:5, loss:0.015679072729489714 , accuracy:97.28833333333333%


**Testing on test dataset:**

In [ ]:
acc=[]
for image,labels in test_loader:
    data=image.reshape(image.shape[0],1,28*28)
    targets=to_one_hot(labels)
    data=data.numpy()
    targets=targets.numpy()
    outputs=net.predict(data)
    acc.append(accuracy(np.argmax(outputs,axis=1),np.argmax(targets,axis=1)))

print(f"test accuracy: {np.mean(acc)*100}")


test accuracy: 96.765525477707
